In [2]:
import pandas as pd
from splink.datasets import splink_datasets
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl

df = splink_datasets.fake_1000

In [4]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on(["first_name", "surname"]),
        block_on(["surname", "dob"]),
        block_on(["first_name", "dob"]),
        block_on(["email", "first_name"]),
    ],
    "comparisons": [
        ctl.name_comparison("first_name",term_frequency_adjustments=True),
        ctl.name_comparison("surname", term_frequency_adjustments=True),
        ctl.date_comparison("dob", cast_strings_to_date=True, invalid_dates_as_null=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],       
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "max_iterations": 10,
    "em_convergence": 0.01
}

linker = DuckDBLinker(df, settings)

linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email",
    ],
    recall=0.7,
)

linker.estimate_u_using_random_sampling(max_pairs=1e6)

training_blocking_rule = block_on(["first_name", "surname"])
training_session_names = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = block_on("dob")
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)



Probability two random records match is estimated to be  0.00333.
This means that amongst all possible pairwise record comparisons, one in 300.13 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 1,664.29 matching pairs
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blo

In [5]:
linker.save_model_to_json("model_f1k.json")

{'link_type': 'dedupe_only',
 'blocking_rules_to_generate_predictions': [{'blocking_rule': '(l."first_name" = r."first_name") AND (l."surname" = r."surname")',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': '(l."surname" = r."surname") AND (l."dob" = r."dob")',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': '(l."first_name" = r."first_name") AND (l."dob" = r."dob")',
   'sql_dialect': 'duckdb'},
  {'blocking_rule': '(l."email" = r."email") AND (l."first_name" = r."first_name")',
   'sql_dialect': 'duckdb'}],
 'comparisons': [{'output_column_name': 'first_name',
   'comparison_levels': [{'sql_condition': '"first_name_l" IS NULL OR "first_name_r" IS NULL',
     'label_for_charts': 'Null',
     'is_null_level': True},
    {'sql_condition': '"first_name_l" = "first_name_r"',
     'label_for_charts': 'Exact match first_name',
     'm_probability': 0.4928215543553728,
     'u_probability': 0.0057935713975033705,
     'tf_adjustment_column': 'first_name',
     'tf_adjustment_weight': 1.0}